# 🚗 Car Detection and Tracking Project

This notebook implements a real-time car detection and tracking system using:
- YOLOv8 for object detection
- DeepSORT for object tracking
- OpenCV for video processing

The system can:
1. Detect vehicles in video frames
2. Track vehicles across frames
3. Count vehicles crossing a specified line
4. Generate a processed video with tracking information

## 📚 Import Required Libraries

First, we import all necessary libraries for the project:
- OpenCV for video processing
- YOLO for object detection
- DeepSORT for object tracking
- NumPy for numerical operations
- Tkinter for file dialog

In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import tkinter as tk
from tkinter import filedialog

## 🎯 Video Selection Function

This function opens a file dialog to select a video file for processing.

In [ ]:
def select_video():
    """Open file dialog to select video file."""
    root = tk.Tk()
    root.withdraw()
    video_path = filedialog.askopenfilename(
        title="Select Video File",
        filetypes=[("Video files", "*.mp4 *.avi *.mov *.mkv"), ("All files", "*.*")]
    )
    if not video_path:
        raise ValueError("No video file selected")
    return video_path

## 📹 Video Writer Setup Function

This function sets up the video writer with the same properties as the input video.

In [ ]:
def setup_video_writer(video_path, cap):
    """Setup video writer with same properties as input video."""
    output_path = video_path.rsplit('.', 1)[0] + '_processed.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    return cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height)), output_path

## 🎨 Tracking Visualization Function

This function draws tracking boxes and IDs on the video frames.

In [ ]:
def draw_tracking_info(frame, track, track_id, box_color=(0, 255, 0)):
    """Draw tracking box and ID on frame."""
    ltrb = track.to_ltrb()
    x1, y1, x2, y2 = map(int, ltrb)
    
    # Draw box
    cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
    
    # Draw ID
    text = f'ID: {track_id}'
    cv2.putText(frame, text, (x1, y1 - 5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    return (x1, y1, x2, y2)

## 🔄 Main Processing Function

This function contains the main processing loop that:
1. Initializes the YOLO model and DeepSORT tracker
2. Processes each frame of the video
3. Detects and tracks vehicles
4. Counts vehicles crossing the line
5. Displays and saves the processed video

In [ ]:
def process_video():
    # Initialize
    video_path = select_video()
    print(f"Processing video: {video_path}")
    
    model = YOLO("yolov8m.pt")
    tracker = DeepSort(max_age=30)
    cap = cv2.VideoCapture(video_path)
    out, output_path = setup_video_writer(video_path, cap)
    
    # Tracking parameters
    line_position = 300
    offset = 10
    counted_ids = set()
    total_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Detect objects
        results = model(frame)[0]
        detections = []
        
        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            if int(class_id) in [2, 3, 5, 7] and score > 0.4:  # Vehicle classes
                detections.append(([x1, y1, x2 - x1, y2 - y1], score, int(class_id)))
        
        # Update tracks
        tracks = tracker.update_tracks(detections, frame=frame)
        
        # Process each track
        for track in tracks:
            if not track.is_confirmed():
                continue
                
            track_id = track.track_id
            x1, y1, x2, y2 = draw_tracking_info(frame, track, track_id)
            cy = int((y1 + y2) / 2)
            
            # Count vehicles crossing the line
            if (line_position - offset) < cy < (line_position + offset):
                if track_id not in counted_ids:
                    counted_ids.add(track_id)
                    total_count += 1
        
        # Draw counting line and display count
        cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)
        cv2.putText(frame, f'Total Vehicles: {total_count}', (20, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        # Display and save frame
        cv2.imshow("Vehicle Tracking", frame)
        out.write(frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Cleanup
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Processing complete! Total vehicles: {total_count}")
    print(f"Output saved: {output_path}")
    return total_count, output_path

## 🚀 Run the Processing

Execute the main processing function to start vehicle detection and tracking. The function will:
1. Open a file dialog to select your video
2. Process the video in real-time
3. Show the tracking visualization
4. Save the processed video
5. Return the total count and output path

In [ ]:
total_count, output_path = process_video()